In [117]:
library(data.table)
library(dplyr)
library(lubridate)

In [64]:
path <- 'DATA/'

In [5]:
customer <- fread(sprintf('%sFINAL_CUSTOMER_DATATHON.csv',path))[,-1]
sales <- fread(sprintf('%sFINAL_SALES_FILE_DATATHON.csv',path))[,-1]
customer_related <- fread(sprintf('%sFINAL_CUSTOMER_RELATED_TABLE_FOR_DATATHON.csv',path))[,-1]
vehicle <- fread(sprintf('%sFINAL_VEHICLE_TABLE_DATATHON.csv',path))[,-1]
buyers <- fread(sprintf('%sFINAL_SIFIR_ARAC_ALANLAR_DATATHON.csv',path))[,-1]
submit_format <- fread(sprintf('%ssample_submission.csv',path))

In [162]:
buyers[,CREATE_DATE:=as.Date(CREATE_DATE)]

In [163]:
buyers %>% 
group_by(CUSTOMER_ID,VEHICLE_ID) %>% 
mutate(samp=n()) %>% 
filter(samp==1) %>%
select(-samp) %>% 
ungroup %>% 
mutate(Y=1)-> buyers 

In [45]:
sales %>%
group_by(SALESFILE_ID) %>% 
arrange(SF_CREATE_DATE,.by_group = T) %>%
mutate(n=1:n()) %>% 
filter(n==max(n)) %>%
select(-n) %>% 
ungroup -> sales

In [89]:
customer_related %>% 
group_by(BASE_CUSTOMER_ID) %>% 
mutate(samp=n()) %>% 
filter(samp<=5) %>%
select(-samp) %>% 
arrange(START_DATE,.by_group = T) %>% 
mutate(sira=1:n()) %>% 
summarise(VEHICLE_ID=VEHICLE_ID[max(sira)],START_DATE=min(START_DATE),END_DATE=END_DATE[max(sira)]) %>% 
mutate(FK_RELATION_STATUS_ID=ifelse(is.na(END_DATE),1,2)) %>%
rowwise %>% 
mutate(TIME_DIFF = diff.Date(c(START_DATE,END_DATE))) %>% 
ungroup -> customer_related

In [135]:
customer_related %>% 
inner_join(vehicle,by='VEHICLE_ID') %>% 
rowwise %>% 
mutate(TRAFFIC_TIME_DIFF = diff.Date(c(START_DATE,TRAFFIC_DATE))) %>% 
ungroup %>% 
mutate(MOTOR_GAS_TYPE = ifelse(MOTOR_GAS_TYPE=='' | MOTOR_GAS_TYPE=='Seçiniz',NA,MOTOR_GAS_TYPE))%>% 
mutate(GEAR_BOX_TYPE = ifelse(GEAR_BOX_TYPE==''  | MOTOR_GAS_TYPE=='Seçiniz',NA,GEAR_BOX_TYPE)) -> customer_related_with_vehicle

In [193]:
customer %>% 
left_join(customer_related_with_vehicle,by='BASE_CUSTOMER_ID') %>% 
left_join(buyers,by=c('VEHICLE_ID','CUSTOMER_ID')) %>% 
mutate(MARITAL_STATUS=ifelse(MARITAL_STATUS=='',NA,MARITAL_STATUS)) %>%
mutate(FK_ADDRESS_COMMUNICATION_CITY=ifelse(FK_ADDRESS_COMMUNICATION_CITY=='',NA,FK_ADDRESS_COMMUNICATION_CITY)) %>% 
mutate(OCCUPATION=ifelse(OCCUPATION=='',NA,OCCUPATION)) %>% 
rowwise %>% 
mutate(BUYER_AGE=diff(c(BIRTH_DATE,year(CREATE_DATE)))) %>% 
ungroup %>% 
mutate(BUYER_AGE=ifelse(is.na(BUYER_AGE),2021 - BIRTH_DATE,BUYER_AGE)) -> final_tablo

In [205]:
submit_format %>% head

Id,Expected
<int>,<lgl>
168254,NA
124747,NA
135493,NA
90543,NA
40421,NA
175788,NA


In [201]:
final_tablo %>% 
anti_join(submit_format,by=c('BASE_CUSTOMER_ID'='Id',)) -> train_set

In [210]:
train_set %>% filter(BASE_CUSTOMER_ID==168254)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,GENDER_ID,MARITAL_STATUS,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION,VEHICLE_ID,⋯,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE,TRAFFIC_TIME_DIFF,CREATE_DATE,Y,BUYER_AGE
<int>,<int>,<chr>,<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,⋯,<date>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<date>,<dbl>,<dbl>


In [217]:
year(NA)-NA

[1] NA

In [202]:
train_set %>% 
mutate(Y=ifelse(is.na(Y),0,Y)) -> train_set



In [221]:
setDT(train_set)
train_set[,Y:=as.factor(Y)]

In [226]:
train_set %>% select(Y) %>% mutate(Y=as.numeric(as.character(Y)))

Y
<dbl>
0
0
0
1
0
0
0
0
0


In [211]:
final_tablo %>% 
inner_join(submit_format,by=c('BASE_CUSTOMER_ID'='Id')) -> test_set